# Experiment // Chain

### Preparation
At the beginning, let's import the most important modules and classes from the MedTagger internal API.

In [1]:
import collections
import numpy as np
from medtagger.database import models
from medtagger.ground_truth.algorithms.majority_voting import MajorityVotingAlgorithm
from medtagger.ground_truth.algorithms.gaussian_mixture_models import GaussianMixtureModelsAlgorithm
from medtagger.ground_truth.algorithms.k_means import KMeansAlgorithm
from medtagger.ground_truth.algorithms.dbscan import DBSCANAlgorithm
from medtagger.ground_truth.parsers.chain import ChainLabelElementParser
from medtagger.ground_truth.generator import DataSetGenerator
from medtagger.ground_truth.quality import figures
from medtagger.ground_truth.quality.user_specificity_sensitivity import \
    compute_specificity_and_sensitivity_for_users

In [2]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [8.0, 6.0]
plt.rcParams['figure.dpi'] = 120

In [3]:
SCAN_IDS = [
    '03647f36-d874-480b-aaec-c2c3b9d80092',
    '40761156-99db-4647-9b72-8e449e6cb54c',
    'b750320e-5664-4eff-b4c7-49e4f43ce6ca',
    '8b5d264d-bc2f-458f-ba68-2a987d55deae',
    '7985ea16-d5a4-4bda-92db-43a3cc074216'
]

### Create Generator instance

To create Ground Truth data set, all you need is to define your data set generator and an algorithm that should be used during generation.

In [4]:
# algorithm = MajorityVotingAlgorithm()
# algorithm = GaussianMixtureModelsAlgorithm()
# algorithm = KMeansAlgorithm()
algorithm = DBSCANAlgorithm()
generator = DataSetGenerator(algorithm)

In [5]:
# PERCENTILE = 0
# PERCENTILE = 50
PERCENTILE = 75

In [6]:
mv_algorithm = MajorityVotingAlgorithm()
mv_generator = DataSetGenerator(algorithm)

In [7]:
parser = ChainLabelElementParser()

### Select Scans for analysis

Then, select all Scans that you would like to analyse.

In [8]:
scans = models.Scan.query.all()
scans_ids = {scan.id for scan in scans}
print(f'Scan IDs: {scans_ids}')

Scan IDs: {'7985ea16-d5a4-4bda-92db-43a3cc074216', 'b750320e-5664-4eff-b4c7-49e4f43ce6ca', '03647f36-d874-480b-aaec-c2c3b9d80092', '8b5d264d-bc2f-458f-ba68-2a987d55deae', '40761156-99db-4647-9b72-8e449e6cb54c'}


### Select Label Elements for analysis

Now, select all Label Elements that should be analysed.

**IMPORTANT:** Currently Data Set Generator assumes only one Label Element per Slice. Don't worry, this will change in the near future.

In [9]:
query = models.ChainLabelElement.query.join(models.Label).join(models.User)
query = query.filter(models.Label.scan_id.in_(scans_ids))
eti_chain_label_elements = query.filter(models.User.last_name == 'ETI').all()
gumed_chain_label_elements = query.filter(models.User.last_name == 'GUMED').all()
expert_chain_label_elements = query.filter(models.User.last_name == 'EXPERT').all()
combined_chain_label_elements = query.filter(models.User.last_name.in_(['ETI', 'GUMED'])).all()
print(f'There are {len(eti_chain_label_elements)} Chain Label Elements from ETI.')
print(f'There are {len(gumed_chain_label_elements)} Chain Label Elements from GUMED.')
print(f'There are {len(combined_chain_label_elements)} Chain Label Elements from ETI & GUMED.')
print(f'There are {len(expert_chain_label_elements)} Chain Label Elements from EXPERT.')
print(f'Example: {eti_chain_label_elements[0]}')

There are 1103 Chain Label Elements from ETI.
There are 422 Chain Label Elements from GUMED.
There are 1525 Chain Label Elements from ETI & GUMED.
There are 130 Chain Label Elements from EXPERT.
Example: <ChainLabelElement: a14fa6c3-d9f4-44e1-a4aa-92d7dac3a3a9: 6 True>


### Generate Ground Truth data set

Take your Label Elements and use generator to generate output Ground Truth annotations for each Slice that took part in the labeling process.

In [10]:
ground_truth_expert = mv_generator.generate(expert_chain_label_elements)
# ground_truth_expert

In [11]:
ground_truth_eti = generator.generate(eti_chain_label_elements)
# ground_truth_eti

In [12]:
gumed_chain_label_elements = [s for s in gumed_chain_label_elements
                              if s.slice_index < len(s.label.scan.slices)]
ground_truth_gumed = generator.generate(gumed_chain_label_elements)
# ground_truth_gumed

In [13]:
combined_chain_label_elements = [s for s in combined_chain_label_elements
                                 if s.slice_index < len(s.label.scan.slices)]
ground_truth_combined = generator.generate(combined_chain_label_elements)
# ground_truth_combined

## [Table 1] Expert labels

In [14]:
labels_per_scan = {}
for slice_id in ground_truth_expert:
    scan_id = models.Scan.query.join(models.Slice).filter(models.Slice.id == slice_id).first().id
    if ground_truth_expert[slice_id] is not None:
        if labels_per_scan.get(scan_id) is None:
            labels_per_scan[scan_id] = 0
        labels_per_scan[scan_id] += 1

In [15]:
slices_per_scan = {}
for scan in models.Scan.query.all():
    scan_id = scan.id
    slices_per_scan[scan_id] = len(scan.slices)

In [16]:
for scan_id in SCAN_IDS:
    print(scan_id, '->', labels_per_scan[scan_id], 'labels,', slices_per_scan[scan_id], 'slices')

03647f36-d874-480b-aaec-c2c3b9d80092 -> 10 labels, 28 slices
40761156-99db-4647-9b72-8e449e6cb54c -> 6 labels, 22 slices
b750320e-5664-4eff-b4c7-49e4f43ce6ca -> 10 labels, 28 slices
8b5d264d-bc2f-458f-ba68-2a987d55deae -> 10 labels, 28 slices
7985ea16-d5a4-4bda-92db-43a3cc074216 -> 26 labels, 50 slices


## [Table 1] Valid vs. Invalid

In [17]:
# eti_valid_labels = {}
# eti_total_labels = {}

# for e in eti_chain_label_elements:
#     scan_id = e.label.scan.id
#     slice_id = e.label.scan.slices[e.slice_index].id
#     if eti_valid_labels.get(scan_id) is None:
#         eti_valid_labels[scan_id] = 0
#         eti_total_labels[scan_id] = 0
#     if ground_truth_expert[slice_id] is not None:
#         eti_valid_labels[scan_id] += 1
#     eti_total_labels[scan_id] += 1

# gumed_valid_labels = {}
# gumed_total_labels = {}

# for e in gumed_chain_label_elements:
#     if e.slice_index >= len(e.label.scan.slices):
#         continue
#     scan_id = e.label.scan.id
#     slice_id = e.label.scan.slices[e.slice_index].id
#     if gumed_valid_labels.get(scan_id) is None:
#         gumed_valid_labels[scan_id] = 0
#         gumed_total_labels[scan_id] = 0
#     if ground_truth_expert[slice_id] is not None:
#         gumed_valid_labels[scan_id] += 1
#     gumed_total_labels[scan_id] += 1

# for scan_id in SCAN_IDS:
#     print(' {:.3f}%'.format(100 * eti_valid_labels[scan_id] / eti_total_labels[scan_id]),
#           '\t{:.3f}%'.format(100 * (eti_total_labels[scan_id] - eti_valid_labels[scan_id]) / eti_total_labels[scan_id]),
#           '\t{:.3f}%'.format(100 * gumed_valid_labels[scan_id] / gumed_total_labels[scan_id]),
#           '\t{:.3f}%'.format(100 * (gumed_total_labels[scan_id] - gumed_valid_labels[scan_id]) / gumed_total_labels[scan_id]))

## [Table 2] Dispersion

In [18]:
# eti_differences = collections.defaultdict(lambda: [])

# for e in eti_chain_label_elements:
#     scan_id = e.label.scan.id
#     slice_id = e.label.scan.slices[e.slice_index].id
#     gt_image = ground_truth_expert.get(slice_id)
#     if gt_image is None:
#         if algorithm.REQUIRE_IMAGE_RESIZE:
#             gt_image = np.zeros((100 * 100))
#         else:
#             gt_image = np.zeros((e.label.scan.width * e.label.scan.height))
#     e_image = parser.convert_to_numpy([e], resize_image=algorithm.REQUIRE_IMAGE_RESIZE)
#     eti_differences[scan_id].append(np.sum(np.abs(e_image - gt_image)) / gt_image.shape[0])


# gumed_differences = collections.defaultdict(lambda: [])

# for e in gumed_chain_label_elements:
#     scan_id = e.label.scan.id
#     slice_id = e.label.scan.slices[e.slice_index].id
#     gt_image = ground_truth_expert.get(slice_id)
#     if gt_image is None:
#         if algorithm.REQUIRE_IMAGE_RESIZE:
#             gt_image = np.zeros((100 * 100))
#         else:
#             gt_image = np.zeros((e.label.scan.width * e.label.scan.height))
#     e_image = parser.convert_to_numpy([e], resize_image=algorithm.REQUIRE_IMAGE_RESIZE)
#     gumed_differences[scan_id].append(np.sum(np.abs(e_image - gt_image)) / gt_image.shape[0])
    
# for scan_id in SCAN_IDS:
#     print(' {:.3f}%'.format(np.std(eti_differences[scan_id]) * 100),
#           '\t{:.3f}%'.format(np.std(gumed_differences[scan_id]) * 100))

## [Table 3] Dispersion in generated Ground Truth

In [19]:
# gt_eti_differences = collections.defaultdict(lambda: 0)
# gt_eti_total = collections.defaultdict(lambda: 0)

# gt_gumed_differences = collections.defaultdict(lambda: 0)
# gt_gumed_total = collections.defaultdict(lambda: 0)

# for slice_id in ground_truth_eti:
#     scan = models.Scan.query.join(models.Slice).filter(models.Slice.id == slice_id).first()
#     scan_id = scan.id
    
#     gt_eti_image = ground_truth_eti.get(slice_id)
#     if gt_eti_image is None:
#         if algorithm.REQUIRE_IMAGE_RESIZE:
#             gt_eti_image = np.zeros((100 * 100))
#         else:
#             gt_eti_image = np.zeros((scan.width * scan.height))
            
#     gt_image = ground_truth_expert.get(slice_id)
#     if gt_image is None:
#         if algorithm.REQUIRE_IMAGE_RESIZE:
#             gt_image = np.zeros((100 * 100))
#         else:
#             gt_image = np.zeros((scan.width * scan.height))

#     gt_eti_differences[scan_id] += np.sum(np.abs(gt_eti_image - gt_image))
#     gt_eti_total[scan_id] += gt_image.shape[0]

# for slice_id in ground_truth_gumed:
#     scan = models.Scan.query.join(models.Slice).filter(models.Slice.id == slice_id).first()
#     scan_id = scan.id
    
#     gt_gumed_image = ground_truth_gumed.get(slice_id)
#     if gt_gumed_image is None:
#         if algorithm.REQUIRE_IMAGE_RESIZE:
#             gt_gumed_image = np.zeros((100 * 100))
#         else:
#             gt_gumed_image = np.zeros((scan.width * scan.height))
    
#     gt_image = ground_truth_expert.get(slice_id)
#     if gt_image is None:
#         if algorithm.REQUIRE_IMAGE_RESIZE:
#             gt_image = np.zeros((100 * 100))
#         else:
#             gt_image = np.zeros((scan.width * scan.height))

#     gt_gumed_differences[scan_id] += np.sum(np.abs(gt_gumed_image - gt_image))
#     gt_gumed_total[scan_id] += gt_image.shape[0]

# for scan_id in SCAN_IDS:
#     print(' {:.3f}%'.format(gt_eti_differences[scan_id] / gt_eti_total[scan_id] * 100),
#           '\t{:.3f}%'.format(gt_gumed_differences[scan_id] / gt_gumed_total[scan_id] * 100))

### [GUMED] Check Users' Specificity and Sensitivity

MedTagger supports easy calculation of Users' Specificity, Sensitivity and Score based on their annotations and Ground Truth data set.

In [20]:
gumed_users = set(element.label.owner for element in gumed_chain_label_elements)
print(f'There are {len(gumed_users)} Users.')

There are 12 Users.


In [21]:
gumed_users_specificity, gumed_users_sensitivity, gumed_users_scores = \
    compute_specificity_and_sensitivity_for_users(algorithm, gumed_users,
                                                  gumed_chain_label_elements,
                                                  ground_truth_gumed)

In [22]:
users_ids = set(u.id for u in gumed_users)
for user_id in users_ids:
    print(f'User #{user_id:03d}: Sensitivity={gumed_users_sensitivity[user_id]:1.5f} '
          f'Specificity={gumed_users_specificity[user_id]:1.5f} Score={gumed_users_scores[user_id]:1.5f}')

User #103: Sensitivity=0.85938 Specificity=1.00000 Score=0.73853
User #104: Sensitivity=0.03125 Specificity=0.98913 Score=0.00042
User #106: Sensitivity=0.14062 Specificity=1.00000 Score=0.01978
User #107: Sensitivity=0.89062 Specificity=0.97826 Score=0.75496
User #109: Sensitivity=0.15625 Specificity=1.00000 Score=0.02441
User #116: Sensitivity=0.85938 Specificity=1.00000 Score=0.73853
User #117: Sensitivity=0.15625 Specificity=0.98913 Score=0.02114
User #118: Sensitivity=0.48438 Specificity=1.00000 Score=0.23462
User #119: Sensitivity=0.04688 Specificity=0.98913 Score=0.00130
User #120: Sensitivity=0.92188 Specificity=0.98913 Score=0.82993
User #121: Sensitivity=0.54688 Specificity=1.00000 Score=0.29907
User #123: Sensitivity=0.92188 Specificity=1.00000 Score=0.84985


In [23]:
# figures.specificity_vs_sensitivity(gumed_users_specificity, gumed_users_sensitivity)

### [GUMED] Compare Labeling Time to Users' Score

You can also check how User's mean Labeling Time outputs in their achieved scores.

In [24]:
# figures.mean_labeling_time_vs_score(gumed_chain_label_elements, gumed_users_scores)

### [ETI] Check Users' Specificity and Sensitivity

MedTagger supports easy calculation of Users' Specificity, Sensitivity and Score based on their annotations and Ground Truth data set.

In [25]:
eti_users = set(element.label.owner for element in eti_chain_label_elements)
print(f'There are {len(eti_users)} Users.')

There are 35 Users.


In [26]:
eti_users_specificity, eti_users_sensitivity, eti_users_scores = \
    compute_specificity_and_sensitivity_for_users(algorithm, eti_users,
                                                  eti_chain_label_elements,
                                                  ground_truth_eti)

In [27]:
users_ids = set(u.id for u in eti_users)
for user_id in users_ids:
    print(f'User #{user_id:03d}: Sensitivity={eti_users_sensitivity[user_id]:1.5f} '
          f'Specificity={eti_users_specificity[user_id]:1.5f} Score={eti_users_scores[user_id]:1.5f}')

User #003: Sensitivity=0.43662 Specificity=1.00000 Score=0.19064
User #005: Sensitivity=0.52113 Specificity=1.00000 Score=0.27157
User #025: Sensitivity=0.46479 Specificity=1.00000 Score=0.21603
User #031: Sensitivity=0.56338 Specificity=0.94118 Score=0.25458
User #033: Sensitivity=0.28169 Specificity=1.00000 Score=0.07935
User #034: Sensitivity=0.47887 Specificity=1.00000 Score=0.22932
User #035: Sensitivity=0.39437 Specificity=1.00000 Score=0.15552
User #036: Sensitivity=0.38028 Specificity=1.00000 Score=0.14461
User #037: Sensitivity=0.45070 Specificity=1.00000 Score=0.20313
User #039: Sensitivity=0.01408 Specificity=1.00000 Score=0.00020
User #040: Sensitivity=0.47887 Specificity=1.00000 Score=0.22932
User #041: Sensitivity=0.45070 Specificity=0.98824 Score=0.19267
User #042: Sensitivity=0.45070 Specificity=1.00000 Score=0.20313
User #043: Sensitivity=0.39437 Specificity=1.00000 Score=0.15552
User #044: Sensitivity=0.74648 Specificity=0.98824 Score=0.53980
User #048: Sensitivity=0.

In [28]:
# figures.specificity_vs_sensitivity(eti_users_specificity, eti_users_sensitivity)

### [ETI] Compare Labeling Time to Users' Score

You can also check how User's mean Labeling Time outputs in their achieved scores.

In [29]:
# figures.mean_labeling_time_vs_score(eti_chain_label_elements, eti_users_scores)

### [COMBINED] Check Users' Specificity and Sensitivity

MedTagger supports easy calculation of Users' Specificity, Sensitivity and Score based on their annotations and Ground Truth data set.

In [30]:
combined_users = set(element.label.owner for element in combined_chain_label_elements)
print(f'There are {len(combined_users)} Users.')

There are 47 Users.


In [31]:
combined_users_specificity, combined_users_sensitivity, combined_users_scores = \
    compute_specificity_and_sensitivity_for_users(algorithm, combined_users,
                                                  combined_chain_label_elements,
                                                  ground_truth_combined)

In [32]:
users_ids = set(u.id for u in combined_users)
for user_id in users_ids:
    print(f'User #{user_id:03d}: Sensitivity={combined_users_sensitivity[user_id]:1.5f} '
          f'Specificity={combined_users_specificity[user_id]:1.5f} Score={combined_users_scores[user_id]:1.5f}')

User #003: Sensitivity=0.41892 Specificity=1.00000 Score=0.17549
User #005: Sensitivity=0.50000 Specificity=1.00000 Score=0.25000
User #025: Sensitivity=0.44595 Specificity=1.00000 Score=0.19887
User #031: Sensitivity=0.54054 Specificity=0.93902 Score=0.22998
User #033: Sensitivity=0.27027 Specificity=1.00000 Score=0.07305
User #034: Sensitivity=0.45946 Specificity=1.00000 Score=0.21110
User #035: Sensitivity=0.37838 Specificity=1.00000 Score=0.14317
User #036: Sensitivity=0.36486 Specificity=1.00000 Score=0.13313
User #037: Sensitivity=0.43243 Specificity=1.00000 Score=0.18700
User #039: Sensitivity=0.01351 Specificity=1.00000 Score=0.00018
User #040: Sensitivity=0.45946 Specificity=1.00000 Score=0.21110
User #041: Sensitivity=0.43243 Specificity=0.98780 Score=0.17660
User #042: Sensitivity=0.43243 Specificity=1.00000 Score=0.18700
User #043: Sensitivity=0.37838 Specificity=1.00000 Score=0.14317
User #044: Sensitivity=0.71622 Specificity=0.98780 Score=0.49565
User #048: Sensitivity=0.

In [33]:
# figures.specificity_vs_sensitivity(combined_users_specificity, combined_users_sensitivity)

### [COMBINED] Compare Labeling Time to Users' Score

You can also check how User's mean Labeling Time outputs in their achieved scores.

In [34]:
# figures.mean_labeling_time_vs_score(combined_chain_label_elements, combined_users_scores)

## [GUMED] Only best labels

In [35]:
gumed_raw_scores = np.array(list(gumed_users_scores.values()))
gumed_median_score = np.percentile(gumed_raw_scores, PERCENTILE)
best_gumed_users_id = [u_id for u_id in gumed_users_scores if gumed_users_scores[u_id] >= gumed_median_score]
best_gumed_users_id

[107, 120, 123]

In [36]:
best_gumed_chain_label_elements = [e for e in gumed_chain_label_elements
                                         if e.label.owner_id in best_gumed_users_id]
ground_truth_gumed_best = generator.generate(best_gumed_chain_label_elements)
# ground_truth_gumed_best

## [ETI] Only best labels

In [37]:
eti_raw_scores = np.array(list(eti_users_scores.values()))
eti_median_score = np.percentile(eti_raw_scores, PERCENTILE)
best_eti_users_id = [u_id for u_id in eti_users_scores if eti_users_scores[u_id] >= eti_median_score]
# best_eti_users_id

In [38]:
best_eti_chain_label_elements = [e for e in eti_chain_label_elements
                                 if e.label.owner_id in best_eti_users_id]
ground_truth_eti_best = generator.generate(best_eti_chain_label_elements)
# ground_truth_eti_best

## [COMBINED] Only best labels

In [39]:
combined_raw_scores = np.array(list(combined_users_scores.values()))
combined_median_score = np.percentile(combined_raw_scores, PERCENTILE)
best_combined_users_id = [u_id for u_id in combined_users_scores if combined_users_scores[u_id] >= combined_median_score]
# best_combined_users_id

In [40]:
best_combined_chain_label_elements = [e for e in combined_chain_label_elements
                                      if e.label.owner_id in best_combined_users_id]
ground_truth_combined_best = generator.generate(best_combined_chain_label_elements)
# ground_truth_combined_best

## [Table 1]

In [41]:
# gumed_valid_labels = {}
# gumed_total_labels = {}

# for e in best_gumed_chain_label_elements:
#     scan_id = e.label.scan.id
#     slice_id = e.label.scan.slices[e.slice_index].id
#     if gumed_valid_labels.get(scan_id) is None:
#         gumed_valid_labels[scan_id] = 0
#         gumed_total_labels[scan_id] = 0
#     if ground_truth_expert[slice_id] is not None:
#         gumed_valid_labels[scan_id] += 1
#     gumed_total_labels[scan_id] += 1

# eti_valid_labels = {}
# eti_total_labels = {}

# for e in best_eti_chain_label_elements:
#     scan_id = e.label.scan.id
#     slice_id = e.label.scan.slices[e.slice_index].id
#     if eti_valid_labels.get(scan_id) is None:
#         eti_valid_labels[scan_id] = 0
#         eti_total_labels[scan_id] = 0
#     if ground_truth_expert[slice_id] is not None:
#         eti_valid_labels[scan_id] += 1
#     eti_total_labels[scan_id] += 1

# for scan_id in SCAN_IDS:
#     print(' {:.3f}%'.format(100 * eti_valid_labels[scan_id] / eti_total_labels[scan_id]),
#           '\t{:.3f}%'.format(100 * (eti_total_labels[scan_id] - eti_valid_labels[scan_id]) / eti_total_labels[scan_id]),
#           '\t{:.3f}%'.format(100 * gumed_valid_labels[scan_id] / gumed_total_labels[scan_id]),
#           '\t{:.3f}%'.format(100 * (gumed_total_labels[scan_id] - gumed_valid_labels[scan_id]) / gumed_total_labels[scan_id]))

In [42]:
# #
# # Dla Dawida
# #
# user_emails = set()

# gumed_differences = collections.defaultdict(lambda: collections.defaultdict(lambda: 0))
# gumed_total = collections.defaultdict(lambda: collections.defaultdict(lambda: 0))

# for e in best_gumed_chain_label_elements:
#     user_id = e.label.owner.email
#     user_emails.add(user_id)
#     scan = e.label.scan
#     scan_id = scan.id
#     slice_id = e.label.scan.slices[e.slice_index].id
#     gumed_image = parser.convert_to_numpy([e], algorithm.REQUIRE_IMAGE_RESIZE)[0]
#     gt_image = ground_truth_expert.get(slice_id)
#     if gt_image is None:
#         if algorithm.REQUIRE_IMAGE_RESIZE:
#             gt_image = np.zeros((100 * 100))
#         else:
#             gt_image = np.zeros((scan.width * scan.height))
    
#     iou = parser.compute_intersection_over_union(gumed_image, gt_image)
#     if not np.isnan(iou):
#         gumed_differences[user_id][scan_id] += iou
#         gumed_total[user_id][scan_id] += 1

# for slice_id in ground_truth_expert:
#     for gumed_id in best_gumed_users_id:
#         user = models.User.query.filter(models.User.id == gumed_id).first()
        
#         gumed_label_element = next((e for e in best_gumed_chain_label_elements
#                                     if e.label.scan.slices[e.slice_index].id == slice_id
#                                       and e.label.owner_id == gumed_id
#                                    ), None)
        
#         if ground_truth_expert.get(slice_id) is not None and gumed_label_element is None:
#             scan_id = models.Scan.query.join(models.Slice).filter(models.Slice.id == slice_id).first().id
#             gumed_differences[user.email][scan_id] += 0
#             gumed_total[user.email][scan_id] += 1
        
# eti_differences = collections.defaultdict(lambda: collections.defaultdict(lambda: 0))
# eti_total = collections.defaultdict(lambda: collections.defaultdict(lambda: 0))

# for e in best_eti_chain_label_elements:
#     user_id = e.label.owner.email
#     user_emails.add(user_id)
#     scan = e.label.scan
#     scan_id = scan.id
#     slice_id = e.label.scan.slices[e.slice_index].id
#     eti_image = parser.convert_to_numpy([e], algorithm.REQUIRE_IMAGE_RESIZE)[0]
#     gt_image = ground_truth_expert.get(slice_id)
#     if gt_image is None:
#         if algorithm.REQUIRE_IMAGE_RESIZE:
#             gt_image = np.zeros((100 * 100))
#         else:
#             gt_image = np.zeros((scan.width * scan.height))
    
#     iou = parser.compute_intersection_over_union(eti_image, gt_image)
#     if not np.isnan(iou):
#         eti_differences[user_id][scan_id] += iou
#         eti_total[user_id][scan_id] += 1


# for slice_id in ground_truth_expert:
#     for eti_id in best_eti_users_id:
#         user = models.User.query.filter(models.User.id == eti_id).first()
        
#         eti_label_element = next((e for e in best_eti_chain_label_elements
#                                     if e.label.scan.slices[e.slice_index].id == slice_id
#                                       and e.label.owner_id == eti_id
#                                    ), None)
        
#         if ground_truth_expert.get(slice_id) is not None and eti_label_element is None:
#             scan_id = models.Scan.query.join(models.Slice).filter(models.Slice.id == slice_id).first().id
#             eti_differences[user.email][scan_id] += 0
#             eti_total[user.email][scan_id] += 1
        
# for user_id in user_emails:
#     for scan_id in SCAN_IDS:
#         if eti_total[user_id][scan_id] > 0:
#             iou = eti_differences[user_id][scan_id] / eti_total[user_id][scan_id] * 100
#         elif gumed_total[user_id][scan_id] > 0:
#             iou = gumed_differences[user_id][scan_id] / gumed_total[user_id][scan_id] * 100
#         else:
#             iou = 0
#         label = models.Label.query.join(models.User).filter(models.User.email == user_id)\
#                    .filter(models.Label.scan_id == scan_id).first()
#         cant_see_anything = False if label is None else label.comment == 'This is an empty Label'
#         timestamp = 0 if label is None else label.labeling_time
#         print('{}\t{}\t{:.3f}%\t{}\t{}'.format(user_id, scan_id, iou, timestamp, cant_see_anything))

## Intersection over Union - między wprowadzonymi etykietami a prawdziwym Ground Truth

In [43]:
gumed_differences = collections.defaultdict(lambda: 0)
gumed_total = collections.defaultdict(lambda: 0)

for e in best_gumed_chain_label_elements:
    scan = e.label.scan
    scan_id = scan.id
    slice_id = e.label.scan.slices[e.slice_index].id
    gumed_image = parser.convert_to_numpy([e], algorithm.REQUIRE_IMAGE_RESIZE)[0]
    gt_image = ground_truth_expert.get(slice_id)
    if gt_image is None:
        if algorithm.REQUIRE_IMAGE_RESIZE:
            gt_image = np.zeros((100 * 100))
        else:
            gt_image = np.zeros((scan.width * scan.height))
    
    iou = parser.compute_intersection_over_union(gumed_image, gt_image)
    if not np.isnan(iou):
        gumed_differences[scan_id] += iou
        gumed_total[scan_id] += 1

for slice_id in ground_truth_expert:
    for gumed_id in best_gumed_users_id:
        user = models.User.query.filter(models.User.id == gumed_id).first()
        
        gumed_label_element = next((e for e in best_gumed_chain_label_elements
                                    if e.label.scan.slices[e.slice_index].id == slice_id
                                      and e.label.owner_id == gumed_id
                                   ), None)
        
        if ground_truth_expert.get(slice_id) is not None and gumed_label_element is None:
            scan_id = models.Scan.query.join(models.Slice).filter(models.Slice.id == slice_id).first().id
            gumed_differences[scan_id] += 0
            gumed_total[scan_id] += 1

eti_differences = collections.defaultdict(lambda: 0)
eti_total = collections.defaultdict(lambda: 0)

for e in best_eti_chain_label_elements:
    scan = e.label.scan
    scan_id = scan.id
    slice_id = e.label.scan.slices[e.slice_index].id
    eti_image = parser.convert_to_numpy([e], algorithm.REQUIRE_IMAGE_RESIZE)[0]
    gt_image = ground_truth_expert.get(slice_id)
    if gt_image is None:
        if algorithm.REQUIRE_IMAGE_RESIZE:
            gt_image = np.zeros((100 * 100))
        else:
            gt_image = np.zeros((scan.width * scan.height))
    
    iou = parser.compute_intersection_over_union(eti_image, gt_image)
    if not np.isnan(iou):
        eti_differences[scan_id] += iou
        eti_total[scan_id] += 1

for slice_id in ground_truth_expert:
    for eti_id in best_eti_users_id:
        user = models.User.query.filter(models.User.id == eti_id).first()
        
        eti_label_element = next((e for e in best_eti_chain_label_elements
                                    if e.label.scan.slices[e.slice_index].id == slice_id
                                      and e.label.owner_id == eti_id
                                   ), None)
        
        if ground_truth_expert.get(slice_id) is not None and eti_label_element is None:
            scan_id = models.Scan.query.join(models.Slice).filter(models.Slice.id == slice_id).first().id
            eti_differences[scan_id] += 0
            eti_total[scan_id] += 1

combined_differences = collections.defaultdict(lambda: 0)
combined_total = collections.defaultdict(lambda: 0)

for e in best_combined_chain_label_elements:
    scan = e.label.scan
    scan_id = scan.id
    slice_id = e.label.scan.slices[e.slice_index].id
    combined_image = parser.convert_to_numpy([e], algorithm.REQUIRE_IMAGE_RESIZE)[0]
    gt_image = ground_truth_expert.get(slice_id)
    if gt_image is None:
        if algorithm.REQUIRE_IMAGE_RESIZE:
            gt_image = np.zeros((100 * 100))
        else:
            gt_image = np.zeros((scan.width * scan.height))
    
    iou = parser.compute_intersection_over_union(combined_image, gt_image)
    if not np.isnan(iou):
        combined_differences[scan_id] += iou
        combined_total[scan_id] += 1

for slice_id in ground_truth_expert:
    for eti_id in best_combined_users_id:
        user = models.User.query.filter(models.User.id == eti_id).first()
        
        eti_label_element = next((e for e in best_combined_chain_label_elements
                                    if e.label.scan.slices[e.slice_index].id == slice_id
                                      and e.label.owner_id == eti_id
                                   ), None)
        
        if ground_truth_expert.get(slice_id) is not None and eti_label_element is None:
            scan_id = models.Scan.query.join(models.Slice).filter(models.Slice.id == slice_id).first().id
            combined_differences[scan_id] += 0
            combined_total[scan_id] += 1

for scan_id in SCAN_IDS:
    print(' {:.3f}%'.format(eti_differences[scan_id] / eti_total[scan_id] * 100),
          '\t{:.3f}%'.format(gumed_differences[scan_id] / gumed_total[scan_id] * 100),
          '\t{:.3f}%'.format(combined_differences[scan_id] / combined_total[scan_id] * 100))

 70.219% 	74.915% 	72.634%
 60.979% 	67.968% 	63.090%
 63.638% 	55.264% 	62.292%
 68.789% 	69.769% 	68.432%
 17.434% 	75.397% 	42.857%


## Intersection over Union - między wyliczonym Ground Truth a prawdziwym Ground Truth

In [44]:
#
# TEMPORARY
#

# gt_eti_differences = collections.defaultdict(lambda: 0)
# gt_eti_total = collections.defaultdict(lambda: 0)

# for slice_id in ground_truth_eti_best:
#     scan = models.Scan.query.join(models.Slice).filter(models.Slice.id == slice_id).first()
#     scan_id = scan.id
    
#     gt_eti_image = ground_truth_eti_best.get(slice_id)
#     if gt_eti_image is None:
#         if algorithm.REQUIRE_IMAGE_RESIZE:
#             gt_eti_image = np.zeros((100 * 100))
#         else:
#             gt_eti_image = np.zeros((scan.width * scan.height))
    
#     gt_image = ground_truth_expert.get(slice_id)
#     if gt_image is None:
#         if algorithm.REQUIRE_IMAGE_RESIZE:
#             gt_image = np.zeros((100 * 100))
#         else:
#             gt_image = np.zeros((scan.width * scan.height))

#     iou = parser.compute_intersection_over_union(gt_eti_image, gt_image)
#     if not np.isnan(iou):
#         gt_eti_differences[scan_id] += iou
#         gt_eti_total[scan_id] += 1

# gt_gumed_differences = collections.defaultdict(lambda: 0)
# gt_gumed_total = collections.defaultdict(lambda: 0)

# for slice_id in ground_truth_gumed_best:
#     scan = models.Scan.query.join(models.Slice).filter(models.Slice.id == slice_id).first()
#     scan_id = scan.id
    
#     gt_gumed_image = ground_truth_gumed_best.get(slice_id)
#     if gt_gumed_image is None:
#         if algorithm.REQUIRE_IMAGE_RESIZE:
#             gt_gumed_image = np.zeros((100 * 100))
#         else:
#             gt_gumed_image = np.zeros((scan.width * scan.height))
    
#     gt_image = ground_truth_expert.get(slice_id)
#     if gt_image is None:
#         if algorithm.REQUIRE_IMAGE_RESIZE:
#             gt_image = np.zeros((100 * 100))
#         else:
#             gt_image = np.zeros((scan.width * scan.height))
    
#     iou = parser.compute_intersection_over_union(gt_gumed_image, gt_image)
#     if not np.isnan(iou):
#         gt_gumed_differences[scan_id] += iou
#         gt_gumed_total[scan_id] += 1

# gt_combined_differences = collections.defaultdict(lambda: 0)
# gt_combined_total = collections.defaultdict(lambda: 0)

# for slice_id in ground_truth_combined_best:
#     scan = models.Scan.query.join(models.Slice).filter(models.Slice.id == slice_id).first()
#     scan_id = scan.id
    
#     gt_combined_image = ground_truth_combined_best.get(slice_id)
#     if gt_combined_image is None:
#         if algorithm.REQUIRE_IMAGE_RESIZE:
#             gt_combined_image = np.zeros((100 * 100))
#         else:
#             gt_combined_image = np.zeros((scan.width * scan.height))
    
#     gt_image = ground_truth_expert.get(slice_id)
#     if gt_image is None:
#         if algorithm.REQUIRE_IMAGE_RESIZE:
#             gt_image = np.zeros((100 * 100))
#         else:
#             gt_image = np.zeros((scan.width * scan.height))
    
#     iou = parser.compute_intersection_over_union(gt_combined_image, gt_image)
#     if not np.isnan(iou):
#         gt_combined_differences[scan_id] += iou
#         gt_combined_total[scan_id] += 1

# for scan_id in SCAN_IDS:
#     print(' {:.3f}%'.format(gt_eti_differences[scan_id] / gt_eti_total[scan_id] * 100),
#           '\t{:.3f}%'.format(gt_gumed_differences[scan_id] / gt_gumed_total[scan_id] * 100),
#           '\t{:.3f}%'.format(gt_combined_differences[scan_id] / gt_combined_total[scan_id] * 100))

## Mean Average Precision @ 0.75 - między wprowadzonymi etykietami a prawdziwym Ground Truth

In [45]:
THRESHOLD = 0.75

gumed_differences = collections.defaultdict(lambda: 0)
gumed_total = collections.defaultdict(lambda: 0)

for e in best_gumed_chain_label_elements:
    scan = e.label.scan
    scan_id = scan.id
    slice_id = e.label.scan.slices[e.slice_index].id
    gumed_image = parser.convert_to_numpy([e], algorithm.REQUIRE_IMAGE_RESIZE)[0]
    gt_image = ground_truth_expert.get(slice_id)
    if gt_image is None:
        if algorithm.REQUIRE_IMAGE_RESIZE:
            gt_image = np.zeros((100 * 100))
        else:
            gt_image = np.zeros((scan.width * scan.height))
    
    iou = parser.compute_intersection_over_union(gt_image, gumed_image)
    iou = 1.0 if np.isnan(iou) else iou
    gumed_differences[scan_id] += 1 if iou >= THRESHOLD else 0
    gumed_total[scan_id] += 1

for slice_id in ground_truth_expert:
    for gumed_id in best_gumed_users_id:
        user = models.User.query.filter(models.User.id == gumed_id).first()
        
        gumed_label_element = next((e for e in best_gumed_chain_label_elements
                                    if e.label.scan.slices[e.slice_index].id == slice_id
                                      and e.label.owner_id == gumed_id
                                   ), None)
        
        if ground_truth_expert.get(slice_id) is not None and gumed_label_element is None:
            scan_id = models.Scan.query.join(models.Slice).filter(models.Slice.id == slice_id).first().id
            gumed_differences[scan_id] += 0
            gumed_total[scan_id] += 1

eti_differences = collections.defaultdict(lambda: 0)
eti_total = collections.defaultdict(lambda: 0)

for e in best_eti_chain_label_elements:
    scan = e.label.scan
    scan_id = scan.id
    slice_id = e.label.scan.slices[e.slice_index].id
    eti_image = parser.convert_to_numpy([e], algorithm.REQUIRE_IMAGE_RESIZE)[0]
    gt_image = ground_truth_expert.get(slice_id)
    if gt_image is None:
        if algorithm.REQUIRE_IMAGE_RESIZE:
            gt_image = np.zeros((100 * 100))
        else:
            gt_image = np.zeros((scan.width * scan.height))
    
    iou = parser.compute_intersection_over_union(gt_image, eti_image)
    iou = 1.0 if np.isnan(iou) else iou
    eti_differences[scan_id] += 1 if iou >= THRESHOLD else 0
    eti_total[scan_id] += 1

for slice_id in ground_truth_expert:
    for eti_id in best_eti_users_id:
        user = models.User.query.filter(models.User.id == eti_id).first()
        
        eti_label_element = next((e for e in best_eti_chain_label_elements
                                    if e.label.scan.slices[e.slice_index].id == slice_id
                                      and e.label.owner_id == eti_id
                                   ), None)
        
        if ground_truth_expert.get(slice_id) is not None and eti_label_element is None:
            scan_id = models.Scan.query.join(models.Slice).filter(models.Slice.id == slice_id).first().id
            eti_differences[scan_id] += 0
            eti_total[scan_id] += 1

combined_differences = collections.defaultdict(lambda: 0)
combined_total = collections.defaultdict(lambda: 0)

for e in best_combined_chain_label_elements:
    scan = e.label.scan
    scan_id = scan.id
    slice_id = e.label.scan.slices[e.slice_index].id
    combined_image = parser.convert_to_numpy([e], algorithm.REQUIRE_IMAGE_RESIZE)[0]
    gt_image = ground_truth_expert.get(slice_id)
    if gt_image is None:
        if algorithm.REQUIRE_IMAGE_RESIZE:
            gt_image = np.zeros((100 * 100))
        else:
            gt_image = np.zeros((scan.width * scan.height))
    
    iou = parser.compute_intersection_over_union(gt_image, combined_image)
    iou = 1.0 if np.isnan(iou) else iou
    combined_differences[scan_id] += 1 if iou >= THRESHOLD else 0
    combined_total[scan_id] += 1

for slice_id in ground_truth_expert:
    for eti_id in best_combined_users_id:
        user = models.User.query.filter(models.User.id == eti_id).first()
        
        eti_label_element = next((e for e in best_combined_chain_label_elements
                                    if e.label.scan.slices[e.slice_index].id == slice_id
                                      and e.label.owner_id == eti_id
                                   ), None)
        
        if ground_truth_expert.get(slice_id) is not None and eti_label_element is None:
            scan_id = models.Scan.query.join(models.Slice).filter(models.Slice.id == slice_id).first().id
            combined_differences[scan_id] += 0
            combined_total[scan_id] += 1

for scan_id in SCAN_IDS:
    print(' {:.3f}%'.format(eti_differences[scan_id] / eti_total[scan_id] * 100),
          '\t{:.3f}%'.format(gumed_differences[scan_id] / gumed_total[scan_id] * 100),
          '\t{:.3f}%'.format(combined_differences[scan_id] / combined_total[scan_id] * 100))

 66.337% 	81.818% 	71.212%
 66.154% 	71.429% 	67.857%
 48.421% 	36.667% 	45.082%
 60.870% 	61.290% 	59.504%
 14.516% 	72.500% 	38.700%


## Mean Average Precision @ 0.75 - między wyliczonym Ground Truth a prawdziwym Ground Truth

In [46]:
#
# TEMPORARY
#

# THRESHOLD = 0.75

# gt_eti_differences = collections.defaultdict(lambda: 0)
# gt_eti_total = collections.defaultdict(lambda: 0)

# for slice_id in ground_truth_eti_best:
#     scan = models.Scan.query.join(models.Slice).filter(models.Slice.id == slice_id).first()
#     scan_id = scan.id
    
#     gt_eti_image = ground_truth_eti_best.get(slice_id)
#     if gt_eti_image is None:
#         if algorithm.REQUIRE_IMAGE_RESIZE:
#             gt_eti_image = np.zeros((100 * 100))
#         else:
#             gt_eti_image = np.zeros((scan.width * scan.height))
    
#     gt_image = ground_truth_expert.get(slice_id)
#     if gt_image is None:
#         if algorithm.REQUIRE_IMAGE_RESIZE:
#             gt_image = np.zeros((100 * 100))
#         else:
#             gt_image = np.zeros((scan.width * scan.height))
    
#     iou = parser.compute_intersection_over_union(gt_eti_image, gt_image)
#     iou = 1.0 if np.isnan(iou) else iou
#     gt_eti_differences[scan_id] += 1 if iou >= THRESHOLD else 0
#     gt_eti_total[scan_id] += 1

# gt_gumed_differences = collections.defaultdict(lambda: 0)
# gt_gumed_total = collections.defaultdict(lambda: 0)

# for slice_id in ground_truth_gumed_best:
#     scan = models.Scan.query.join(models.Slice).filter(models.Slice.id == slice_id).first()
#     scan_id = scan.id

#     gt_gumed_image = ground_truth_gumed_best.get(slice_id)
#     if gt_gumed_image is None:
#         if algorithm.REQUIRE_IMAGE_RESIZE:
#             gt_gumed_image = np.zeros((100 * 100))
#         else:
#             gt_gumed_image = np.zeros((scan.width * scan.height))
    
#     gt_image = ground_truth_expert.get(slice_id)
#     if gt_image is None:
#         if algorithm.REQUIRE_IMAGE_RESIZE:
#             gt_image = np.zeros((100 * 100))
#         else:
#             gt_image = np.zeros((scan.width * scan.height))

#     iou = parser.compute_intersection_over_union(gt_gumed_image, gt_image)
#     iou = 1.0 if np.isnan(iou) else iou
#     gt_gumed_differences[scan_id] += 1 if iou >= THRESHOLD else 0
#     gt_gumed_total[scan_id] += 1

# gt_combined_differences = collections.defaultdict(lambda: 0)
# gt_combined_total = collections.defaultdict(lambda: 0)

# for slice_id in ground_truth_combined_best:
#     scan = models.Scan.query.join(models.Slice).filter(models.Slice.id == slice_id).first()
#     scan_id = scan.id

#     gt_combined_image = ground_truth_combined_best.get(slice_id)
#     if gt_combined_image is None:
#         if algorithm.REQUIRE_IMAGE_RESIZE:
#             gt_combined_image = np.zeros((100 * 100))
#         else:
#             gt_combined_image = np.zeros((scan.width * scan.height))
    
#     gt_image = ground_truth_expert.get(slice_id)
#     if gt_image is None:
#         if algorithm.REQUIRE_IMAGE_RESIZE:
#             gt_image = np.zeros((100 * 100))
#         else:
#             gt_image = np.zeros((scan.width * scan.height))

#     iou = parser.compute_intersection_over_union(gt_combined_image, gt_image)
#     iou = 1.0 if np.isnan(iou) else iou
#     gt_combined_differences[scan_id] += 1 if iou >= THRESHOLD else 0
#     gt_combined_total[scan_id] += 1

# for scan_id in SCAN_IDS:
#     print(' {:.3f}%'.format(gt_eti_differences[scan_id] / gt_eti_total[scan_id] * 100),
#           '\t{:.3f}%'.format(gt_gumed_differences[scan_id] / gt_gumed_total[scan_id] * 100),
#           '\t{:.3f}%'.format(gt_combined_differences[scan_id] / gt_combined_total[scan_id] * 100))